<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de Twenty One Pilots

In [21]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('/content/drive/MyDrive/NLP/twenty-one-pilots-songs.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,I wish I found some better sounds
1,No one's ever heard
2,I wish I had a better voice
3,That sang some better words
4,I wish I found some chords


In [23]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2048


### 1 - Preprocesamiento

In [24]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [25]:
# Demos un vistazo
sentence_tokens[:2]

[['i', 'wish', 'i', 'found', 'some', 'better', 'sounds'],
 ['no', "one's", 'ever', 'heard']]

### 2 - Crear los vectores (word2vec)

In [26]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [27]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [28]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [29]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2048


In [30]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 440


### 3 - Entrenar el modelo generador

In [32]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=30,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 45585.421875
Loss after epoch 1: 44307.421875
Loss after epoch 2: 42938.015625
Loss after epoch 3: 42395.515625
Loss after epoch 4: 41281.359375
Loss after epoch 5: 39353.78125
Loss after epoch 6: 39374.015625
Loss after epoch 7: 37602.71875
Loss after epoch 8: 37207.46875
Loss after epoch 9: 36277.375
Loss after epoch 10: 35647.4375
Loss after epoch 11: 35409.53125
Loss after epoch 12: 34378.3125
Loss after epoch 13: 33265.625
Loss after epoch 14: 32923.9375
Loss after epoch 15: 32367.375
Loss after epoch 16: 32137.25
Loss after epoch 17: 31227.625
Loss after epoch 18: 31548.9375
Loss after epoch 19: 31553.25
Loss after epoch 20: 30776.5
Loss after epoch 21: 31206.5625
Loss after epoch 22: 30457.3125
Loss after epoch 23: 29728.6875
Loss after epoch 24: 30681.9375
Loss after epoch 25: 30762.8125
Loss after epoch 26: 30178.0
Loss after epoch 27: 30085.6875
Loss after epoch 28: 29713.875
Loss after epoch 29: 29525.9375


(211284, 394740)

### 4 - Ensayar

In [49]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["car"], topn=5)

[('somebody', 0.9050265550613403),
 ('radio', 0.8665909767150879),
 ("what's", 0.8336623907089233),
 ('asleep', 0.8042196035385132),
 ('catch', 0.7938840985298157)]

In [41]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["jumpsuit"], topn=5)

[('cover', 0.9486323595046997),
 ('high', 0.8199109435081482),
 ('takes', 0.7709957361221313),
 ('off', 0.7656264305114746),
 ('heavy', 0.7464791536331177)]

In [50]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=5)

[('on', -0.1022009551525116),
 ('take', -0.12049267441034317),
 ("i'm", -0.12197503447532654),
 ('vibes', -0.14276574552059174),
 ('they', -0.14782799780368805)]

In [45]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["da"], topn=5)

[('la', 0.9789245128631592),
 ('fall', 0.8014169335365295),
 ('place', 0.7724950313568115),
 ('away', 0.7453992962837219),
 ('asleep', 0.7440023422241211)]

In [48]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["stressed"], topn=5)

[('pushing', 0.8640446662902832),
 ("we're", 0.8408600091934204),
 ('sideways', 0.8032404780387878),
 ('driving', 0.7847074270248413),
 ('cause', 0.7739291787147522)]

In [54]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["leave"], topn=5)

[('these', -0.016639262437820435),
 ('fallen', -0.06000002473592758),
 ('much', -0.0720355212688446),
 ('lyrics', -0.07481648772954941),
 ('too', -0.09026876837015152)]

In [46]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: ignored

### 5 - Visualizar agrupación de vectores

In [55]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [56]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

